# Estimating Carbon Emissions Using Census Data

Created by: Derek Robinson <br>
Last Updated: January 10, 2021

### Learning Objectives

In this Jupyter Notebook you will learn or review how to
<ul>
    <li> display shapefile data (.shp) in R and gain exposure to R; </li>
    <li> gain an understanding of how a simple method for carbon estimation can be conducted; </li>
    <li> estimate carbon emissions for different census units</li>  
</ul>

### Assignment Completion

For those in GEOG 653/GEMCC 630 @UWaterloo, you will need to do the following:

<ul>
    <li> Complete this Jupyter Notebook.</li>
    <ul>
        <li> To advance to the next text cell or execute the code in a cell hold <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">Shift</mark></font> and press <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">Enter</mark></font> </li>
        <li> If there is a method for which you would like to know more about its parameters or how it works then you can place your cursor on the method and press <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">Shift + Tab</mark></font> and a help box will appear that you can expand and scroll through.</li>
    </ul>
    <li> Answer the questions at the bottom of this notebook in a separate document (e.g., MS WORD) and submit your answers to the corresponding assignment dropbox on LEARN. Answering the questions may require you to </li>
    <ul>
        <li> copy results from your notebook, </li>
        <li> take screen captures of your notebook output, </li>
        <li> change values in the Jupyter Notebook and re-execute cells/code to obtain new results, </li>
        <li> and summarize and critique literature. </li>
    </ul>
    <li> <b>While you may discuss course content with your classmates, you are to complete the assignment individually.</b></li>
</ul>

### Problem Statement & Data

In this Notebook we are going to use census data to estimate carbon emissions within the Kitchener-Cambridge-Waterloo (KCW) region.

The province of Ontario has been called on by the Federal Government to provide an estimate of carbon emissions from household energy use. The government recognizes that households in different income brackets have different energy-use patterns and that the distribution of the population across income brackets is heterogeneous within provinces and among different provinces (see figure below).

<p style="text-align:center;"><img src="Figure2.jpg" style="width: 75%;"/></p>

The following publication provides an example of how census data can be broken down to estimate retail spending, but the same approach can be used to estimate carbon emissions. This publication also contains the figure shown above:

https://doi.org/10.1080/13658816.2018.1423684
 
**While the methods below are used only for the KCW region, the census data may be acquired for all of Canada and therefore estimates could be created at a national level.** 

### Getting started
Sometimes when running functions and code in R you will receive warnings. These are displayed in pink in Jupyter Notebook following the execution of a cell. The warnings arise for a variety of reasons like a parameter was not specified so a default will be used. In creating these notebooks I have reviewed the warnings and they do not affect the results. The next line of code turns them off so that you are not confused or worried about them. However, if you would like to see them you may restart the notebook and comment the following cell code or skip executing it.

In [ ]:
#options(warn=-1)
# command to turn warnings on
options(warn=0)

<b>Setup and Reading in Shapefiles</b><br>
In R, we can perform statistical and spatial analyses using shapefile data. Working with these data requires using functions from software packages developed by different groups of people, which are referred to as libraries. 

The first library we need is `rgdal` which contains several functions for handling geo-spatial data. The first time you use a library you have to install it with the `install.packages()` function. After the package is installed you only need to call the `library()` function to gain access to it in a Notebook or R script. In this notebook, the install commands have been commented out using `#` symbols so that the compiler will not interpret the code/text. If you receive an error related to rgdal or other packages then you may need to uncomment the lines and execute the cell again.

In [ ]:
#install.packages("rgdal", repos="http://R-Forge.R-project.org")
library(rgdal)

To import a shapefile we will use the `readOGR()` function from `rgdal`. This function allows us to define a shapefile as a usable object in R. In addition to our SHIFT + TAB shortcut, you can get the details about any function in R by typing a question mark `?` in front of the function at a command line. Try executing the next cell to learn more about the `readOGR` function. If the output from any command is too cumbersome to scroll through, then you can click on the blue bar to the left of that output (looks like a blue scoll bar), which will minimize the output.

In [ ]:
?readOGR

`readOGR()` has two required inputs. The first input is the data source name (dsn) of the file. For us this will be `DA_KCW_2011.shp`. 

The second input requirement is layer, which is the layer name `DA_KCW_2011`. Note that the specifications for dsn and layer may vary with the rgdal function being used, but for now we can use them as text descriptors of our file. I stored my data in a different folder so I specified the location of the shapefile relative to the location of the script using `Data/DA_KCW_2011.shp`. Data specifies the folder where my data is located. Then I can import the DA_KCW_2011.shp file using:

In [ ]:
census <- readOGR("Data/DA_KCW_2011.shp", "DA_KCW_2011")

When working with functions that require parameters or variable inputs you can be more explicit about the inputs by using keyword declarations like dsn= and layer= in front the inputs as follows: 

`census <- readOGR(dsn="Data/DA_KCW_2011.shp", layer="DA_KCW_2011")`

Adding these additional details can be a bit annoying, but doing so can be very helpful if you’re curious about the type of input parameter or variable and how it links to the function. It is also useful if you want to share your code or you do not want to forget about how the code works when you come back at a later date.

In [ ]:
census <- readOGR(dsn="Data/DA_KCW_2011.shp", layer="DA_KCW_2011")

Now we have an object in R (`census`) that represents our shapefile. R stores data in what are referred to as a `data.frame` and our vector data (i.e., polygons for the census units) are stored in a type of data frame called a `spatial polygons data frame`. 

Data frames are often thought of as being similar to tables of data. If you type `print (census)` and execute the command then the data frame or table representing the census shapefile will print out. Note, columns can be split because the console or output window may not be large enough to display all the data. 

This next step may take a minute to complete so be patient.

In [ ]:
print(census)

Next, visualize the spatial data of the census units. To do this we can use the `plot()` command, which is a simple command in R that is used to view data in a non-tabular format. Our data may take a long time to plot if you are using your home computer. The plot function is used for an easy way to view the data. It will not create a nice looking map. If you wish to create a nice looking map then you should look into the `ggplot2` libraries. However, for now it is good to veryify the data.

In [ ]:
plot(census)

## Estimating Carbon Emissions

Our conceptual process to estimating carbon emissions is the following:
* Calculate the average household energy (GJ) usage per fuel type by income bracket
* Convert energy estimates to estimates of carbon emissions
* Calculate the number of households in each income bracket in each census unit
* Multiply the number of households in each income bracket by carbon estimates and sum the results for each census (i.e., deriving the total average CO<sub>2</sub> for each census unit)
* Map estimates CO<sub>2</sub> emissions for each census unit spatially.


First, take a look at the following table, produced by Statistics Canada, that provides information on household energy use, by fuel type and by province (Table 3-1; 2011).

http://www.statcan.gc.ca/pub/11-526-s/2013002/t003-eng.htm

These data describe the total energy use collectively for all households across Ontario and identifies the percentage of this total attributed to each of five energy/fuel types (i.e., electricity, natural gas, oil, wood and wood pellets, and propane). Statistics Canada also publishes information on the average household energy use, by household income brackets for Ontario. 

http://www.statcan.gc.ca/pub/11-526-s/2013002/t010-eng.htm


### Calculate the average gigajoules per fuel type for households in each income bracket

To make this calculation, multiply the fuel-type proportion of the energy use for all Ontario households by the average household energy use in each income bracket. For example, you might identify that 

* 28% of all household energy use in Ontario is derived from electricity
* average household incomes less than $20,000 use 61 gigajoules of energy (on average) 

Multiplying 0.28 by 61 produces a result of 17.08, which enables us to very crudely estimate that households with an average income less than $20,000 use 17.08 gigajoules of electricity annually on average. 

First, replace the `<fueltype>` with the remainder of the five fueltypes. 

In [ ]:
fuelType <- c("Electricity", "<fueltype>", "<fueltype>", "<fueltype>", "<fueltype>")

Now create a list of corresponding household energy uses for each fueltype. For example, the Statistics Canada table identifies that 28% of all household energy in Ontario is derived from electricity. So the first value in our list is given as `0.28`. Replace the 0.00 values with the remaining values from Statistics Canada.

In [ ]:
percentON <- c(0.28, 0.00, 0.00, 0.00, 0.00)

You can view the values of your variables `fuelType` and `percentON` by typing those variable names into a cell and executing the cell. Feel free to try this. In the next cell block we will use a combine function `cbind()` to combine our two variables so that they can be viewed as a table.

In [ ]:
percON_fueltypeTable <- cbind(fuelType, percentON)
percON_fueltypeTable

Now we will create a similar table that identifies the income brackets and the average gigajoules of energy per household within those income brackets. Take note that the income brackets provided by Statistics Canada, related to energy consumption, are a little different from those reported in the Census data. 

In the next cell block, add the remaining income bracket labels to our variable `incBrackets`.

In [ ]:
incBrackets <- c("< $20,000", "0", "0", "0", "0", "0", "0", "Not Stated", "All Households")

Now we need to create a variable with the average gigajoules per household `aveGjHH` produced by households in each of those income brackets. Finish the list below and be sure to ensure that the label and the corresponding value are in the same position in each list.

In [ ]:
aveGjHH <- c(61, 00, 00, 00, 00, 00, 00, 00, 00)

Similar to how we combined the data above to create a table. We will do the same here, using the `cbind()` function to create a table of income brackets and corresponding average gigajoules per household (`incBracTable`). 

In [ ]:
incBracTable <- cbind(incBrackets, aveGjHH)
incBracTable

To finish calculating the average gigajoules per household by fuel type, we have to split our total energy values by the different fuel types. So we have to derive 28% of the ave gigajoules per household for those households with an income \<$20000 and then repeat the process for each of the other income brackets.

R provides a way for us to do this fairly easily. We can multiply our list of average gigajoules per household (`aveGjHH`) by the percent of energy consumption by all households for that energy type. To get just percent of energy as electricity, we can specfiy the first element in our `percentON` variable using brackets as follows `percentON[1]`. Then we can repeat the process for the other energy types.

In [ ]:
elec <- aveGjHH * percentON[1]
natGas <- aveGjHH * percentON[2]
oil <- aveGjHH * percentON[3]
wood <- aveGjHH * percentON[4]
propane <- aveGjHH * percentON[5]

Now we can add to our existing table `incBracTable` our new estimates of energy consumption by combining it with our new variables we just created as follows:

In [ ]:
incBracTable <- cbind(incBracTable, elec, natGas, oil, wood, propane)
incBracTable

Nice work. We're almost done. 

### Conversion Factors

You have calculated the average household energy consumption (in GJ) for households in multiple income brackets and by different fuel types. Different fueltypes have different CO<sub>2</sub> emissions and different income brackets use different amounts of fuel. Now we will use conversion factors to estimate the Metric Tons of CO<sub>2</sub> per gigajoule for each fuel type. 

I have provided some conversion factors below for you, but these are not very accurate. 

Lets create an array of conversion factors for each fuel type. 

In [ ]:
metricTco2 <- c(0.004, 0.05,0.069,0.005, 0.06)

Then we can apply each of these conversion factors to our estimates of energy consumption for each fueltype using a similar process as we did above. In this case `elec` contains the estimated GJ produced by the average household in each income bracket. If we multiply this list of values against our conversion of GJ to metric tonnes of CO<sub>2</sub> then we will derive a variable that stores the amount of CO<sub>2</sub> from electricity for the average household in each income bracket.

In [ ]:
elecCo2 <- elec*metricTco2[1]
natGasCo2 <- natGas*metricTco2[2]
oilCo2 <- oil*metricTco2[3]
woodCo2 <- wood*metricTco2[4]
propaneCo2 <- propane*metricTco2[5]
totalAveCo2 <- elecCo2 + natGasCo2 + oilCo2 + woodCo2 + propaneCo2

Then we will combine our new variables of CO<sub>2</sub> estimates into a table called `tableco2`.

In [ ]:
co2Table <- cbind(incBrackets, elecCo2, natGasCo2, oilCo2, woodCo2, propaneCo2, totalAveCo2)
co2Table

Great work! You now have estimates of the amount of CO<sub>2</sub> produced by each fueltype by the average household in each income bracket.

### Spatial distribution of CO<sub>2</sub> emissions by households

To map our estimates of CO<sub>2</sub> household emissions, we have to identify the number of households in each income bracket and multiple that value against our average household CO<sub>2</sub> emissions estimate for each census unit. However, the income brackets used for reporting energy consumption are not the same as the Census income brackets. Therefore we have to sum up the number of households in each Census income bracket to match the same income brackets for reporting energy consumption.

I have started the process, whereby `numHHbrac1`represents the number of households in the first income bracket. Also note that `numHHbrac1` is an array of values with each element corresponding to a different census unit.

You will need to look at the census Dictionary for the remaining variable names for numHHbrac3-6.

In [ ]:
numHHbrac1 <- as.numeric(census$TC1375) + as.numeric(census$TC1376) + as.numeric(census$TC1377) + as.numeric(census$TC1378)
numHHbrac2 <- as.numeric(census$TC1379) + as.numeric(census$TC1380)
numHHbrac3 <- as.numeric(census$TC0000) + as.numeric(census$TC0000)
numHHbrac4 <- as.numeric(census$TC0000) + as.numeric(census$TC0000)
numHHbrac5 <- as.numeric(census$TC0000) + as.numeric(census$TC0000)
numHHbrac6 <- as.numeric(census$TC0000)

Now we can multiply the number of households in each bracket (in each census unit) by the total average carbon emissions (`totalAveCo2`) estimated for the average household in that income bracket.

In [ ]:
numHHbrac1Co2 <- numHHbrac1 * totalAveCo2[1]
numHHbrac2Co2 <- numHHbrac2 * totalAveCo2[2]
numHHbrac3Co2 <- numHHbrac3 * totalAveCo2[3]
numHHbrac4Co2 <- numHHbrac4 * totalAveCo2[4]
numHHbrac5Co2 <- numHHbrac5 * totalAveCo2[5]
numHHbrac6Co2 <- numHHbrac6 * totalAveCo2[6]

Lets sum the CO<sub>2</sub> values in each income bracket to obtain an estimate of total CO<sub>2</sub> emissions for each census unit.

In [ ]:
totalHHCo2 <- numHHbrac1Co2 + numHHbrac2Co2 + numHHbrac3Co2 + numHHbrac4Co2 + numHHbrac5Co2 + numHHbrac6Co2
#totalHHCo2

Now we can merge our new CO<sub>2</sub> estimates with our spatial data by creating a new column, which involves using the `[ ]` and providing a label `CO2`. Then assigning that column the values we just calculated via `totalHHCo2`.

In [ ]:
census[["CO2"]]=totalHHCo2

Finally, we can map our CO<sub>2</sub> estimates. Here we will use the `tmap` library just to keep things simple. If you are running this notebook from your own machine then you will likely need to remove the comment character `#` and install the `tmap` library. You can look up the specifics for the tmap library online. However, here I have added a title with a newline using `\n` and I have specified the number of categories to classify the data as ten `n = 10`. If you see any warnings, you can just ignore them.

In [ ]:
#install.packages("tmap")
library(tmap)
tmap_mode("plot")
tm_shape(census) + tm_polygons("CO2", title="CO2 emissions \nby Census Unit", n = 10)

### Exporting SpatialPointsDataFrame to Shapefile

It may be the case you want to work with the data we have generated in a geographic information system like ArcGIS Pro, QGIS, or Saga. To so so, we have to export the data we just created.
To export our R data to a shapefile, or to create a shapefile of our R SpatialPolygonsDataFrame, we will use the function `writeOGR()` from the `rgdal` package, which has the following four required inputs: 

1. the object we want to convert to a shapefile, which can only be of a spatial data frame type
2. the dsn
3. layer similar to `readOGR()` that we used earlier
4. and the driver, that specifies the type of file to be created, which in our case is ESRI Shapefile. 

We will also use one optional input called `overwrite_layer` and assign it to `TRUE` so that we can rerun the code without getting a 'file already exists' error message. Optional variables are preceded by the name of the variable and an equals sign `overwrite_layer=TRUE` to prevent R from confusing it with other optional variables. Execute the command in the next cell to generate a new shapefile of the census data with our CO2 estimates included.

In [ ]:
writeOGR(census, "censusWCO2.shp", "censusWCO2", driver="ESRI Shapefile", overwrite_layer = TRUE)

We can assign the created shapefile to a new object using the assignment operator `<-`, but at this point we are just going to save it to our hard disk. The `census` input to the writeOGR function tells the function what data frame we are going to use. The first string “censusWCO2.shp” is the dsn (i.e., data source name) output filename which I have specified as census with CO2. The second string is the layer name, which I typically just use the same name as the filename but without the file extension, which in this case is `censusWCO2`. The output should be stored in your working directory, which, since we did not originally define a working directory, should be the location where this notebook resides.

Open your new file in your favourite geographic information system (GIS) or at least one you have handy. I opened it up in QGIS (http://www.qgis.org/) since I am working on my MacBook. Open the attribute table and you should see your two x and y attributes. You could also navigate to the shapefile properties and review the spatial referencing information.

## Assignment Questions

For Questions 2, 4, and 5, please write your response using complete sentences and paragraphs and include a reference list with the references cited throughout the text. The writing should <b>not be in bullet format</b> and should be at a quality for which you would feel confident sharing with a potential employer in an interview or submitting as part of a manuscript for publication. Submit a document with your name, assignment number, and responses to the corresponding LEARN dropbox once completed.

### Question 1. 
Alternative data are provided with your assignment for two additional census units (census subdivisions and census divisions). Now that the methods have been formalized within the Jupyter Notebook. Changes to different values or the extent of the household carbon emissions estimates can be performed with relative ease. To see how this can take place, rerun the notebook with each of these other census units and submit a copy of your two maps. Consider how changes in the aggregation of the census data may alter your spatial representation of carbon emissions (you do not have to write up anything about this issue, but it is important to be cognizant of aggregation effects and what is known as the modifiable areal unit problem).

(4 marks)

### Question 2.

Conduct a literature review to find 6 or more journal articles that provide information about values used in the Jupyter Notebook to estimate and map household carbon emissions using census data. These data may include, but are not limited to different values for household energy consumption, household carbon emissions, proportions of fuel types, and income brackets. Assume you are using the literature to justify the use of different values.

Maximum length = 1 page

(10 marks)

### Question 3

Using your results from Question 2. Produce a new map of estimated carbon emissions and describe how the patterns are similar or are different. Your answer can be qualitative, or you can describe the differences quantitatively (e.g., total difference in emissions, differing numbers of high and low clusters). 

(10 Marks)


### Question 4

Use the literature to describe alternative methods to estimating household carbon emissions than that used in this notebook OR modifications that would improve the approach implemented in this notebook. If you’re feeling up to it, you may modify the notebook to demonstrate how a different method would work and how the mapped estimates would differ.

Maximum length = 1 page

(10 marks)


### Question 5

Write up three limitations to the methodology used in this notebook. Use literature to support your arguments.

Maximum length = 0.5 pages

(5 marks)




## You have reached the end of this notebook.